In [6]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split
from surprise import SVD, NMF

In [7]:
sys.path.insert(1, '../src')
from Collaborative_recommender import CollaborativeRecommender

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

In [10]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

In [12]:
df_all_15.head(5)

,Unnamed: 0,fit,title,also_buy,image_x,price,asin,overall,reviewerID,reviewText,item_text,title_nlp
0,0,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A1J5VYG8O0YEMB,work advertis work advertis,black mountain,black mountain product resist band set door an...
1,1,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,4.0,AYMDRRCOPKTCF,four star product arriv prompt describ,black mountain,black mountain product resist band set door an...
2,2,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,4.0,A3AX86YSUHLELL,great resist band top qualiti okay resist band...,black mountain,black mountain product resist band set door an...
3,3,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A12MJJT1RL2S7K,perfect transport resist band perfect cross fi...,black mountain,black mountain product resist band set door an...
4,4,NaN,Black Mountain Products Resistance Band Set wi...,"['1612431712', 'B01AVDVHTI', 'B002YQUP7Q', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,$17.32,7245456313,5.0,A33BKACRGR3H38,awesom great band crosstrain dont like lift gy...,black mountain,black mountain product resist band set door an...


In [13]:
df_all_15.shape

(227509, 12)

In [14]:
df_all_15.columns

Index(['Unnamed: 0', 'fit', 'title', 'also_buy', 'image_x', 'price', 'asin',
       'overall', 'reviewerID', 'reviewText', 'item_text', 'title_nlp'],
      dtype='object')

In [15]:
df_collab = df_all_15[['reviewerID',"asin","overall"]]

In [16]:
df_collab.head()

,reviewerID,asin,overall
0,A1J5VYG8O0YEMB,7245456313,5.0
1,AYMDRRCOPKTCF,7245456313,4.0
2,A3AX86YSUHLELL,7245456313,4.0
3,A12MJJT1RL2S7K,7245456313,5.0
4,A33BKACRGR3H38,7245456313,5.0


In [17]:
df_collab.shape

(227509, 3)

In [22]:
UM = pd.pivot_table(df_collab,index='reviewerID', columns='asin', values='overall')

In [ ]:
UM

In [44]:
b = (~np.isnan(UM.values)).sum()

In [47]:
b/(b+a)*100

0.034218338461255854

In [18]:
df_collab.describe()

,overall
count,227509.000000
mean,4.474311
std,0.955971
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


## Using surprise to develop a collaborative filter (NMF based method)

In [19]:
reader = surprise.Reader(rating_scale=(1,5))
recom = CollaborativeRecommender(df_collab,reader,NMF)

Use grid search to find the best paramaters for the model

In [20]:
# param_grid = {'n_factors': [4,8,12,16,20,24,32], 'lr_all': [0.002, 0.005], 'n_epochs': [5, 10]}
param_grid = {'n_factors': [8,15,23,30,37,42], 'n_epochs': [5, 10]}
recom.grid(param_grid,cv=3)
recom.fit()


Accuracy for the best model

In [22]:
recom.accuracy()

RMSE: 0.6931


0.6931348160779834

Top n prediction for a user

In [ ]:
bucket = 'recommender-system-amazon' 
csv_buffer = StringIO()
df_collab.sm.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_colab_sm.csv').put(Body=csv_buffer.getvalue())


In [33]:
df_collab['asin'].values

array(['0899332757', '0899332757', '0899332757', ..., 'B01HJDGJ1E',
       'B01HJDGJ1E', 'B01HJGAJ9O'], dtype=object)

In [27]:
UM.columns

Index(['0899332757', '0899333257', '0971100764', '3843518912', '7245456275',
       '7245456313', '7245456453', '8804850086', '9641766805', 'B00000IURU',
       ...
       'B01HJ234E0', 'B01HJ4DN08', 'B01HJ4EYS8', 'B01HJ8YC0Y', 'B01HJA241G',
       'B01HJA7KWE', 'B01HJDGJ1E', 'B01HJDHNX2', 'B01HJDZ34I', 'B01HJGAJ9O'],
      dtype='object', name='asin', length=56351)